# Stable Baselines3 - Car Racing

In [1]:
!pip install python-dateutil
!pip install pyyaml
!pip install packaging
!pip install ipython-genutils
!pip install jinja2
!pip install requests
!pip install jinja2
!pip install markupsafe
!pip install requests
!pip install moviepy
!apt-get install swig
!pip install box2d-py -q
!pip install gym
!pip install -q "stable-baselines3[extra]"

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to 

## Importações

In [2]:
import gym
import numpy as np
import base64
import os
from pathlib import Path
from IPython import display as ipythondisplay
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

### Importando Algoritmo e Política

#### Função Auxiliar

In [3]:
def evaluate(model, num_episodes=10, deterministic=True, initial_state=0):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    vec_env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = vec_env.reset()
        while not done:
            action, _states = model.predict(obs, deterministic=deterministic)
            obs, reward, done, info = vec_env.step(action)
            episode_rewards.append(reward)
        all_episode_rewards.append(sum(episode_rewards))
    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)
    return mean_episode_reward

### Criando Ambiente

In [4]:
env = gym.make("CarRacing-v2", continuous=False)
env = DummyVecEnv([lambda: env])
display(env.observation_space)
display(env.action_space)

/home/melot/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Box(0, 255, (96, 96, 3), uint8)

Discrete(5)

### Avaliando agente não treinado

In [5]:
eval_env = gym.make("CarRacing-v2", continuous=False)
rl_model = PPO("CnnPolicy", env)
mean_reward, std_reward = evaluate_policy(
    rl_model,
    eval_env,
    n_eval_episodes=10,
    deterministic=True,
)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/home/melot/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/melot/.local/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/home/melot/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


mean_reward=-84.78 +/- 3.386792418068155


## Treina o agente

### Inicialização

In [6]:
quantities = [2e4, 5e4, 1e5, 2e5, 4e5]
qtd_teste = 10

### Execução

In [ ]:
for quantity in quantities:
    print(quantity)
    rl_model = PPO("CnnPolicy", env)
    rl_model.learn(total_timesteps=quantity)
    rl_model.save(f"./{quantity}k_car_racing_PPO")
    evaluate(rl_model, num_episodes=qtd_teste, initial_state=quantity)
    del rl_model
    env.reset()
    print()

20000.0
Mean reward: 1.4454018 Num episodes: 10

50000.0
Mean reward: 338.59998 Num episodes: 10

100000.0
Mean reward: 488.6187 Num episodes: 10

200000.0
Mean reward: 583.3642 Num episodes: 10

400000.0


## Avalia o agente treinado

### Avaliação com visualização (jupyter notebook)

In [ ]:
for quantity in quantities:
    eval_env = gym.make("CarRacing-v2", continuous=False, render_mode="human")
    eval_env = DummyVecEnv([lambda: eval_env])
    rl_model = PPO.load(f"./{quantity}k_car_racing_PPO", eval_env)
    mean_reward, std_reward = evaluate_policy(rl_model, eval_env, n_eval_episodes=1, render=True)
    print(f"{quantity}: mean_reward={mean_reward:.2f} +/- {std_reward}")
    eval_env.close()

#### Resultados

### Prepare video recording

In [ ]:
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
def show_videos(video_path="", prefix=""):
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

We will record a video using the [VecVideoRecorder](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder) wrapper, you will learn about those wrapper in the next notebook.

In [ ]:
def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make("CarRacing-v2", render_mode="rgb_array", continuous=False)])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )
    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)
    eval_env.close()

### Inicializar modelo salvo e grava video

In [ ]:
for quantity in quantities:
    model = PPO.load(f"{quantity}k_car_racing_PPO", env)
    record_video(f"{quantity}k_car_racing_PPO", model, video_length=600, prefix=f"ppo-{quantity}k_car_racing_PPO")

In [ ]:
show_videos(video_path="videos", prefix="ppo")